In [56]:
import pandas as pd
import numpy as np
import nltk, re, time, json
from tqdm import tqdm

In [176]:
# Training dataset
df = pd.read_csv("data/train", sep='\t', names=["s_idx", "word_type", "pos"])

# The starting POS tag distribution from training dataset
start_tag_distributions = df[ df['s_idx'] == 1 ]["pos"].value_counts(normalize=True)

# All the possible pos tags from training dataset
pos_tags = df['pos'].unique()

In [177]:
# Load vocab list
vocab = pd.read_csv('vocab.txt', sep='\t', names=['word_type', 'vocab_idx', 'occurrence'])
# unique words in vocab for unk word assignment
vocab_list = set(vocab["word_type"].unique().flatten())

In [150]:
# Do something to the number strings
df[ df['word_type'].str.match('\d\.?\d*')==True ].value_counts()

s_idx  word_type  pos
3      1988       CD     34
8      10         CD     33
11     30         CD     32
9      8          CD     32
4      1          CD     29
                         ..
13     2.3        CD      1
       2.4225     CD      1
       2.45       CD      1
       2.5        CD      1
110    50,000     CD      1
Length: 14013, dtype: int64

# Taks 1  - Generate vocab.txt

In [62]:
# Use value_count() on word_type column to get the occurrences of word types
unique_words = df['word_type'].value_counts()
unique_words = unique_words.reset_index()
vocab = pd.DataFrame(unique_words)

# Add index column
vocab["vocab_idx"] = vocab.index

# Rename and rearrange columns
vocab.columns = ['word_type', 'occurrence', 'vocab_idx']
vocab = vocab[['word_type', 'vocab_idx', 'occurrence']]

# set unknown word threshold, and drop word types with low frequency from vocab
threshold = 2
unks = vocab[ vocab['occurrence'] < threshold ]
vocab = vocab[ vocab['occurrence'] >= threshold ]

# sum the occurrences of all the unknown words and put it in the first row of vocab
unks_df = pd.DataFrame([["<unk>", 0, unks.occurrence.sum()]], columns = ['word_type', 'vocab_idx', 'occurrence'])
vocab = unks_df.append(vocab, ignore_index=True)

# reindex and update vocab_idx values
vocab["vocab_idx"] = vocab.index

# unique words in vocab for unk word assignment
vocab_list = set(vocab["word_type"].unique().flatten())

print("Threshold: {}, Vocab size: {}, <unk> occurrence: {}".format(threshold, vocab.shape[0], unks.occurrence.sum()))

Threshold: 2, Vocab size: 23183, <unk> occurrence: 20011


<p>no preprocess: Threshold: 2, Vocab size: 23183, unk occurrence: 20011</p>
<p>lowercase: Threshold: 2, Vocab size: 21158, unk occurrence: 17401</p>
<p></p>

In [63]:
# vocab.to_csv('vocab'+'_'+str(threshold)+'.txt', sep='\t', header=False, index=False)
vocab.to_csv('vocab.txt', sep='\t', header=False, index=False)

#### What is the selected threshold for unknown words replacement?
<p> The selected threshold is 2 </p>
    
#### What is the total size of your vocabulary
<p> The size of my vocabulary is 23183 </p>

#### what is the total occurrences of the special token < unk > after replacement?
<p> The total occurrences of unk is 20011 </p>

# Taks 2  - Model Learning

In [115]:
# for calculating count(tag)
pos_distributions = df['pos'].value_counts().to_dict()

# keep track of the transitions and emissions
transitions = {}
emissions = {}

# keep track of the keys in transitions and emissions
e_keys = set()
t_keys = set()

prev_pos = None

for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    cur_word = row['word_type']
    cur_pos = row['pos']
    # replace low frequent word with <unk>
    if cur_word not in vocab_list:
        cur_word = "<unk>"
        
    e_key = cur_pos+","+cur_word
    if e_key in e_keys:
        emissions[e_key]+=(1/pos_distributions[cur_pos])
    else:
        emissions[e_key]=(1/pos_distributions[cur_pos])
        e_keys.add(e_key)
    # skip transition for the first word in a sentence
    if row['s_idx'] != 1:
        t_key = prev_pos+","+cur_pos
        if t_key in t_keys:
            transitions[t_key]+=(1/pos_distributions[prev_pos])
        else:
            transitions[t_key]=(1/pos_distributions[prev_pos])
            t_keys.add(t_key)
    prev_pos = cur_pos

    
print("Transitions: {}, Emissions:{}".format(len(transitions), len(emissions)))

# put transitions and emissions into hmm_model
hmm_model = {"transition": transitions, "emission" : emissions}
# store hmm into a file
with open('hmm.json', 'w') as fp:
    json.dump(hmm_model, fp)

100%|██████████| 912095/912095 [00:39<00:00, 23141.63it/s]


Transitions: 1351, Emissions:30303


#### How many transition and emission parameters in your HMM?
<p> 1351 pairs of transitions,  23373 pairs of emissions </p>

# Task 3: Greedy Decoding withHMM

In [178]:
# hmm_model = json.load(open('hmm'+'_'+str(threshold)+'.json',))
hmm_model = json.load(open('hmm.json',))

In [198]:
# greedy decoding function, return accuracy and output_array according to parameters
# dataset: df, output: boolean, is_dev: boolean
def greedy(dataset, output, is_dev):
    # for accuracy computation
    total, correct = 0, 0
    
    # keep track of prev state
    prev_tag = None
    
    # output file
    output_array = []
    
    # Iterate over dataset, print progress
    for i, row in tqdm(dataset.iterrows(), total=dataset.shape[0]):
        word = row['word_type']
        if output: 
            output_row = [row['s_idx'], word]
        
        if is_dev:
            target = row['pos']
        
        if word not in vocab_list:
            word = "<unk>"

        pred = None
        max_s = -1

        # For starting word:
        if row["s_idx"] == 1:
            # for each possible starting tag    
            for tag in start_tag_distributions.keys():
                t,e = 0,0
                # t(s_j)
                t = start_tag_distributions[tag]
                # e(x|s) = 0 if the emission is not seen in training data
                if tag+','+word in hmm_model['emission']:
                    e = hmm_model['emission'][tag+','+word]
                s = e*t
                # argmax s and update predication
                if s > max_s:
                    max_s = s
                    pred = tag  
        # For the rest of the sentence
        else:
            # for each possible tag
            for tag in pos_tags:
                t,e = 0,0
                
                # t(s_j|s_j-1) = 0 if the transition is not seen in training data
                if prev_tag+','+tag in hmm_model['transition']:
                    t = hmm_model['transition'][prev_tag+','+tag]
                    # e(x|s) = 0 if the emission is not seen in training data
                    if tag+','+word in hmm_model['emission']: 
                        e = hmm_model['emission'][tag+','+word]
                s = e*t
                # argmax s and update predication
                if s > max_s:
                    max_s = s
                    pred = tag        
        if(output):   
            output_row.append(pred)
            # Add empty row between sentences
            if row["s_idx"] == 1:
                output_array.append([None, None, None])
            output_array.append(output_row)
            
        # remember the current predication as the prev_tag for next observation
        prev_tag = pred
        # increment correct if predicted right
        if is_dev:
            if pred == target:
                correct +=1
        total +=1

    # output , remove the first Nan
    return round(correct/total*100, 3), output_array

In [163]:
# Dev dataset evaluation
dev_df = pd.read_csv("data/dev", sep='\t', names=["s_idx", "word_type", "pos"])

is_output, is_dev = False, True

accuracy, output = greedy(dev_df, is_output, is_dev)

print("Dev Dataset Accuracy: {}%".format(accuracy))

100%|██████████| 131768/131768 [00:12<00:00, 10244.23it/s]

Dev Dataset Accuracy: 93.503%


In [195]:
# Test dataset predication
test_df = pd.read_csv("data/test", sep='\t', names=["s_idx", "word_type"])

is_output, is_dev = True, False

accuracy, output = greedy(test_df, is_output, is_dev)

100%|██████████| 129654/129654 [00:11<00:00, 11590.63it/s]


In [197]:
# Store predications to greedy.out
# Remove 1st empty line from output
Predictions = np.array(output[1:])
greedy_output_df = pd.DataFrame(Predictions, columns = ["s_idx", "word_type", "pos"])
greedy_output_df.to_csv('greedy.out', sep='\t', header=False, index=False)

#### What is the accuracy on the dev data?
<p> 93.50% </p>

# Task 4: Viterbi Decoding withHMM

In [181]:
# Viterbi Algorithm Implementation, take dataset contains one sentences from pos 1 to pos m
# dataset: df, output: boolean, is_dev: boolean
def viterbi(dataset):
    # pi table
    pi = {}
    # path table
    paths = {}
    # best tag for each position
    best = None
    # position 
    j = 0
    
    # Iterate over dataset, print progress
#     for i, row in tqdm(dataset.iterrows(), total=dataset.shape[0]):
    for i, row in dataset.iterrows():
        j = row['s_idx']
        # the <unk> tag was already replaced in input dataset
        word = row['word_type']
        # Keep track of the pi value for each pos tag
        # {"DT" : 0.001  }
        pi_pos = {}
        # keep track of the path for each pos in pi_pos
        pi_path = {}
    
        if j == 1:
            # reset pi for each sentence
            pi = {}
            paths = {}
            
            for tag in pos_tags:
                # initialize transition and emission
                t,e = 0,0
                # start word t = start tag distribution
                if tag in start_tag_distributions.keys():
                    t = start_tag_distributions[tag]
                    if tag+','+word in hmm_model['emission']:
                        e = hmm_model['emission'][tag+','+word]
                # record pi for each starting tag
                pi_pos[tag] = e*t
                # start the path for each starting tag
                pi_path[tag] = [tag]
            # record the pi and paths at each position j
            pi[j] = pi_pos
            paths[j] = pi_path
                  
        else:
            for cur_tag in pos_tags:
                pi_pos[cur_tag] = -1

                # pi[j-1][prev_tag], t(cur_tag|prev_tag), e(word|cur_tag)
                for prev_tag in pos_tags:
                    prev_pi = pi[j-1][prev_tag]
                    # Skip to the next prev_tag if pi[j-1, prev_tag] = 0
                    cur_pi = 0
                    # cur_pi = 0 if pre_pi = 0 so only consider the non zero prev_pi
                    if prev_pi != 0:
                        t,e = 0,0
                        # if transition exists
                        if prev_tag+','+cur_tag in hmm_model['transition']:
                                t = hmm_model['transition'][prev_tag+','+cur_tag]
                                # if emission exists
                                if cur_tag+','+word in hmm_model['emission']: 
                                    e = hmm_model['emission'][cur_tag+','+word]

                        cur_pi = prev_pi*e*t
                    
                    # update pi at each position as well as update the path to get to the best pi
                    if cur_pi > pi_pos[cur_tag]:
                        pi_pos[cur_tag] = cur_pi
                        pi_path[cur_tag] = paths[j-1][prev_tag][:]
                        pi_path[cur_tag].append(cur_tag)
                        
            # record the pi and paths at each position j           
            pi[j] = pi_pos
            paths[j] = pi_path
        
    # with j being the last position, get the best tag with highest pi value
    best = max(pi[j], key=pi[j].get) 
    # return the sequence that led to the best tag
#     print(pi)
#     print(paths)
    return paths[j][best]

In [214]:
def viterbi_decoder(dataset, is_output, is_dev):

    # Represent each sentence key = word, value = pos, e.g. {"word", 1}
    sentence = []
    targets = []

    # Compute accuracy
    correct = 0
    total = 0
    
    # Output
    output = []

    for i, row in tqdm(dataset.iterrows(), total=dataset.shape[0]):
        # increment total
        total += 1
        # Get word and sentence_idx of each row
        j = row['s_idx']
        if is_dev:
            target = row['pos']
        word = row['word_type']
        if word not in vocab_list:
                word = "<unk>"
        w_row = [j, word]

        if j == 1:
            # Process previous complete sentence stored in dict sentence
            if sentence:     
                sen_df = pd.DataFrame(sentence, columns =['s_idx', 'word_type'])
                #print(sen_df)
                # get predictions from viterbi
                preds = np.array(viterbi(sen_df))
                # compare preds and target, increment correct
                if is_dev:
                    targets = np.array(targets)
                    correct += np.sum(preds == targets)
                # Generate output 
                for i in range(len(preds)):
                    o_row = sentence[i][:]
                    o_row.append(preds[i])
                    output.append(o_row)
                output.append([None,None,None])
                # empty sentence and targets for the next sentence
                sentence = []
                targets = []
        sentence.append(w_row)
        if is_dev:
            targets.append(target)

    # Process the last complete sentece from input df        
    if sentence:
        sen_df = pd.DataFrame(sentence, columns =['s_idx', 'word_type'])
        #print(sen_df)
        # get predictions from viterbi
        preds = np.array(viterbi(sen_df))
        # compare preds and target, increment correct
        if is_dev:
            targets = np.array(targets)
            correct += np.sum(preds == targets)
        # Generate output
        for i in range(len(preds)):
            o_row = sentence[i][:]
            o_row.append(preds[i])
            output.append(o_row)
#         output.append([None,None,None])
    accuracy = round(correct/total*100, 2)
    
    return accuracy, output

In [209]:
dev_df = pd.read_csv("data/dev", sep='\t', names=["s_idx", "word_type", "pos"])            
accuracy, output = viterbi_decoder(dev_df, False, True)
print("Dev Dataset Accuracy: {}%".format(accuracy))

100%|██████████| 131768/131768 [01:42<00:00, 1291.63it/s]

Dev Dataset Accuracy: 94.77%


In [215]:
test_df = pd.read_csv("data/test", sep='\t', names=["s_idx", "word_type", "pos"])            
accuracy, output = viterbi_decoder(test_df, True, False)
# Store predications to greedy.out
# Remove 1st empty line from output
Predictions = np.array(output)
greedy_output_df = pd.DataFrame(Predictions, columns = ["s_idx", "word_type", "pos"])
greedy_output_df.to_csv('viterbi.out', sep='\t', header=False, index=False)

100%|██████████| 129654/129654 [01:36<00:00, 1347.09it/s]


threshold 2, viterbi dev accuracy: 94.77%

#### What is the accuracy on the dev data?
<p> 94.77% </p>

In [183]:
# ### Viterbi algorithm test
# d = {'DT': 0.8, 'NN': 0.2, 'VB': 0}
# start_tag_distributions = pd.Series(data=d, index=['DT', 'NN', 'VB'])

# trans = {}

# tags = ["DT", "NN", "VB"]
# pos_tags = ["DT", "NN", "VB"]

# for tag in tags:
#     for tag2 in tags:
#         trans[tag+","+tag2] = 0

# trans['DT,DT'] = 0
# trans['DT,NN'] = 0.9
# trans['DT,VB'] = 0.1
# trans['NN,DT'] = 0
# trans['NN,NN'] = 0.5
# trans['NN,VB'] = 0.5
# trans['VB,DT'] = 0.5
# trans['VB,NN'] = 0.5
# trans['VB,VB'] = 0

# words = ["the", "fans", "watch", "the", "race"]
# vocab_list = words


# emis = {}

# for tag in tags:
#     for word in words:
# #         print("emis['{},{}'] =".format(tag,word) )
#         emis[tag+","+word] = 0
        
# emis['DT,the'] = 0.2
# emis['DT,fans'] = 0
# emis['DT,watch'] = 0
# emis['DT,race'] = 0
# emis['NN,the'] = 0
# emis['NN,fans'] = 0.1
# emis['NN,watch'] = 0.3
# emis['NN,race'] = 0.1
# emis['VB,the'] = 0
# emis['VB,fans'] = 0.2
# emis['VB,watch'] = 0.15
# emis['VB,race'] = 0.3

# hmm_model = {'transition': trans, 'emission': emis}

# dev_df = pd.read_csv("test/dev", sep='\t', names=["s_idx", "word_type", "pos"])
# #dev_df = dev_df.head(37)

# print(words)
# viterbi(dev_df)